# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# download necessary packages
import nltk
nltk.download(['punkt', 'stopwords', 'words', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'wordnet'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# import libraries

import pandas as pd
import numpy as np
import pickle

from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
# import sqlite3
# import sqlalchemy
# import matplotlib.pyplot as plt
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from sklearn.model_selection import GridSearchCV
# from sklearn.datasets import make_multilabel_classification
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report


# nltk.download(['punkt', 'wordnet'])

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("DisasterResponse", con=engine)
df.dropna(axis=0, subset=df.columns[4:],inplace=True)
df = df # drop any columns will all 0
X = df['message']  # Message Column
Y = df.iloc[:, 4:] # Classification label
Y = Y.loc[:, (Y != 0).any(axis=0)]
Y = Y.loc[:, (Y != 1).any(axis=0)] #drop any column with all 0 or all 1
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
Y.shape

(26386, 35)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    This function is to split the text into meaningful words (tokens).
    Arg:
        text: a string contains information
    return:
        tokens: a list of meaningful words (tokens)
    """
    # normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenization
    words = word_tokenize(text)

    # remove stop words
    words_no_stop = [w for w in words if w not in stopwords.words("english")]
    
    # tag each word with part of speech
    # sentence = pos_tag(words)
    
    # recognize named entities in text
    # tree = ne_chunk(sentence)
    
    # Lemmatization
    Lemmed = [WordNetLemmatizer().lemmatize(w) for w in words_no_stop]
    #Lemmed_v = [WordNetLemmatizer().lemmatize(w, pos = 'v') for w in Lemmed ]

    return Lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Pipleine 0: Logistic Regression

pipeline_logistic = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=LogisticRegression()))
])


# Pipleine 1: Random Forest Classifier

pipeline_rfc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])


# Pipeline 2: Naive Bayes classifier

pipeline_nbc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(MultinomialNB()))
    ])


# Pipleine 3: Adaboost Classifier 

pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:

# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [8]:
Y.mean()

related                   0.775032
request                   0.171038
offer                     0.004586
aid_related               0.415144
medical_help              0.079550
medical_products          0.049989
search_and_rescue         0.027477
security                  0.017850
military                  0.032707
water                     0.063822
food                      0.112029
shelter                   0.088759
clothing                  0.015539
money                     0.022967
missing_people            0.011408
refugees                  0.033351
death                     0.045630
other_aid                 0.131282
infrastructure_related    0.064769
transport                 0.045933
buildings                 0.050974
electricity               0.020390
tools                     0.006026
hospitals                 0.010725
shops                     0.004548
aid_centers               0.011711
other_infrastructure      0.043773
weather_related           0.278292
floods              

In [9]:

# Fit the Logistic Regression
pipeline_logistic.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))])

In [10]:

# Fit the Naive Bayes classifier 
pipeline_nbc.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ssifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
type(pipeline_nbc)

sklearn.pipeline.Pipeline

In [12]:
# for logistic regression
y_pred = pipeline_logistic.predict(X_test)
# y_pred = pd.DataFrame(y_pred, columns=y_test.columns)
# y_pred.head()


In [13]:
y_pred.shape

(5278, 35)

In [14]:
y_test.shape

(5278, 35)

In [15]:
y_pred == y_test.values

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ...,  True,  True, False],
       ..., 
       [ True, False,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [16]:
def report_model_performance(test, pred):
    i = 0
    for col in test:
        print ('Feature {}: {}'.format(i+1, col))
        print(classification_report(test[col], pred[:, i]))
        i+=1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))
    
    return

        


In [17]:
report_model_performance(y_test, y_pred)

Feature 1: related
             precision    recall  f1-score   support

          0       0.73      0.43      0.54      1228
          1       0.84      0.95      0.89      3999
          2       0.00      0.00      0.00        51

avg / total       0.80      0.82      0.80      5278

Feature 2: request
             precision    recall  f1-score   support

          0       0.92      0.97      0.94      4362
          1       0.82      0.57      0.67       916

avg / total       0.90      0.90      0.90      5278

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5260
          1       0.00      0.00      0.00        18

avg / total       0.99      1.00      0.99      5278

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.80      0.84      0.82      3119
          1       0.75      0.69      0.72      2159

avg / total       0.78      0.78      0.78      5278

Feature 5

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
pipeline_logistic.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd34a0e8950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
             penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
             verbose=0, warm_start=False),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtyp

In [19]:
parameters_logistic = {'tfidf__use_idf': (True, False),  'clf__estimator__max_iter': [50, 100, 500, 1000],}

cv_logistic = GridSearchCV(pipeline_logistic, param_grid = parameters_logistic)
cv_logistic

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__max_iter': [50, 100, 500, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
type(cv_logistic)

sklearn.model_selection._search.GridSearchCV

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv_logistic.fit(X_train, y_train)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv_logistic, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.